# Packed Json Graph - Steps
A Packed Json Graph is a Json structure that has minimal hierarchy of data and has no graph specific information such as separate edges list

In [1]:
from neo4j import GraphDatabase
import cypher_utils as ctl
from cypher_utils import cypher
import utils as utl

ctl.open("bolt://localhost:7687", "neo4j", "test")

In [2]:
packed_json = utl.load_json("../data/zigbee_packed.json")
len(packed_json)

3

In [3]:
cypher("match (n) detach delete n")

In [4]:
def stringify_props(name,node,packed_json):
    props = f"name:'{name}'"
    for key,prop_val in node.items():
        if(key in packed_json):#This is a relationship not a property, so skip in node creation
            continue
        if(type(node[key]) == str):
            props += f", {key}:'{node[key]}'"
        else:
            props += f", {key}:{node[key]}"
    return props

def stringify_relations(name,node,packed_json):
    relations = ""
    for key,prop_val in node.items():
        if(key in packed_json):
            assert(isinstance(prop_val,list))
            for target in prop_val:
                relations += f"\n (`{name}`)-[:relatesto]->(`{target}`),"
    return relations

In [ ]:
def json_packed_to_cypher(packed_json):
    cmd = "CREATE "
    #Nodes
    for Label,Labels_map in packed_json.items():
        for name,node in Labels_map.items():
            props = stringify_props(name,node,packed_json)
            cmd += f"\n (`{name}`:{Label} {{{props}}}),"
    #Relationships
    for Label,Labels_map in packed_json.items():
        for name,node in Labels_map.items():
            cmd += stringify_relations(name,node,packed_json)
    cmd = cmd[:-1]+";"
    return cmd

In [ ]:
print(cmd)

In [26]:
utl.save_file("zigbee_generic.cyp",cmd)

5759 bytes saved in zigbee_generic.cyp


In [24]:
cypher(cmd)